# BTTH09: Ẩn dữ liệu khả nghịch trên ảnh bằng phương pháp DE (phần 3)

TODO: Võ Phương Hòa - 1412192

---

## 1. Cách làm bài và nộp bài

**Làm bài**

Bạn sẽ làm trực tiếp trên file notebook này; trong file, mình đã để từ `TODO` để cho biết những chỗ mà bạn cần phải làm (trong đó, `TODO` đầu tiên là bạn phải ghi họ tên và MSSV vào phần đầu của file). Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.

*Mục tiêu chính ở đây là học, học một cách chân thật; nếu bạn được 5 điểm nhưng bạn làm bài một cách chân thật thì vẫn tốt hơn nhiều so với 10 điểm mà không chân thật. Bạn có thể thảo luận với các bạn khác; nhưng bài làm phải là của chính bạn, dựa trên sự hiểu của chính bạn, lời văn là của chính bạn, code là của chính bạn. Bạn không nên đọc bài làm của các bạn năm trước. Nếu bị phát hiện gian lận thì bạn sẽ bị 0 điểm cho toàn bộ phần thực hành.*

**Nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart & Run All` để restart và chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Kernel` - `Restart & Run All` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, trong thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) bạn đặt: file `Ex09-ReversibleDataHiding-Img-DE3.ipynb`, và các file dữ liệu mà mình đính kèm (`cover.bmp`, `msg.txt`); rồi nén thư mục `MSSV` này lại và nộp ở link trên moodle.

## 2. Import

In [1]:
%matplotlib inline
import numpy as np
import bitarray
from scipy.misc import imread, imsave
import matplotlib.pyplot as plt
import zlib
import math
# You can also import other things ...

## 3. Hàm nhúng

Ở bài này, bạn sẽ cài đặt hoàn chỉnh phương pháp DE. Ở hàm nhúng, bạn cần `print` ra:

- Kích thước của các tập EZ + EN, CN, NC.
- Số lượng h = 1, -2 của tập CN.
- Chiều dài của location map trước và sau khi nén.
- Chiều dài của msg bits (trước khi thêm '100...').
- Cho biết có nhúng được hay không?

### 3.1. Hàm kiểm tra h expandable

In [2]:
def is_expandable(value, avr):
    value = abs(2 * value + (value >= 0))
    num_1 = 2 * (255 - avr)
    num_2 = 2 * avr + 1
    if min(value, num_1, num_2) == value:
        return True
    return False

### 3.2. Hàm kiểm tra h changeable

In [3]:
def is_changeable(value, avr):
    value = abs(math.floor(float(value)/2) * 2 + (value >= 0))
    num_1 = 2 * (255 - avr)
    num_2 = 2 * avr + 1
    if min(value, num_1, num_2) == value:
        return True
    return False

### 3.3. Hàm nhúng

In [4]:
def embed_DE(cover_img_file, msg_file, stego_img_file):
    """
    Embeds a message into a cover image using the DE (Difference Expansion) method.
    
    Parameters
    ----------
    cover_img_file : string
        The name of the cover img file.
    msg_file : string
        The name of the message file.
    stego_img_file : string
        The name of the stego img file.
    
    Returns
    -------
    result : bool
        result = True if it's possible to embed; False otherwise.
    """
    # TODO
    # 1. Read cover img file
    cover_img = imread('cover.bmp')
    
    # 2. Read msg file and process
    # 2.1. Read msg file
    f = open(msg_file, 'r')
    msg_text = f.read()
    
    # 2.2. Convert msg_text to bitstring
    msg_bit = bitarray.bitarray()
    msg_bit.frombytes(msg_text)  # P
    
    # 3. The date-embedding DE algorithm
    # 3.1. Step 1: calculate the difference values h and the integer average l
    H = []
    L = []
    stego_img = cover_img.reshape(cover_img.size)
    for i in range(0, len(stego_img), 2):
            h = int(stego_img[i]) - int(stego_img[i + 1])
            l = (int(stego_img[i]) + int(stego_img[i + 1])) / 2
            H.append(h)
            L.append(l)
    
    # 3.2. Step 2: partition difference values into four sets (EZ, EN, CN, NC)
    # EZ: contains all expandable h = 0 and expandable h = -1
    # EN: contains all expandable h not in EZ
    # CN: contains all changeable h not in (EZ and EN)
    # NC: contains all nonchangeable h
    EZ = []
    EN = []
    CN = []
    NC = []
    for i in range(len(H)):
        value = H[i]
        avr = L[i]
        if is_expandable(value, avr):
            if value == 0 or value == -1:
                EZ.append(i)
            else:
                EN.append(i)
        elif is_changeable(value, avr):
            CN.append(i)
        else:
            NC.append(i)
    
    print 'Kích thước của EZ + EN:', len(EZ) + len(EN)
    print 'Kích thước của CN:', len(CN)
    print 'Kích thước của NC:', len(NC)

    # 3.3. Step 3 and 4: create a location map and collect original LSB values
    location_map = []
    LSBs = []

    count = 0
    for i in range(len(H)):
        if (i in EZ) or (i in EN):
            location_map.append(1)
        else:
            location_map.append(0)
            # step 4
            if (i not in NC):
                if H[i] != 1 and H[i] != -2:
                    LSBs.append(H[i] % 2)
                else:
                    count += 1
    

    LSBs = bitarray.bitarray(LSBs) # C
    lm = bitarray.bitarray(location_map)
    zlib_lm = bitarray.bitarray()
    zlib_lm.frombytes(zlib.compress(lm.tobytes())) # zlib_lm is L
    
    print 'Số lượng h = 1 và h = -2 của CN:', count
    print 'Chiều dài lm trước và sau khi nén:', len(lm), ',', len(zlib_lm)
    print 'Chiều dài của msg_bit ban đầu:', len(msg_bit)

    # 3.4. Step 5: data embedding by replacement
    len_B = len(zlib_lm) + len(LSBs) + len(msg_bit)
    len_bit = len(EZ) + len(EN) + len(CN) - len_B
    if len_bit <= 0:
        print 'Không ẩn dữ liệu được!'
        return False
    msg_bit += '1' + '0' * (len_bit - 1)
    bitstream = zlib_lm + LSBs + msg_bit  # B = L + C + P
    
    i = 0
    j = -1
    while (i < len(bitstream)):
        j += 1
        #if (j in EZ) or (j in EN):
        if (j not in CN) and (j not in NC):
            H[j] = 2 * H[j] + bitstream[i]
            i += 1
        elif (j in CN):
            H[j] = 2 * (math.floor(float(H[j])/2)) + bitstream[i]
            i += 1
        
    # 3.5. Step 6: compute new value base on new different value
    for i in range(len(H)):
        j = i * 2
        stego_img[j] = L[i] + (int(H[i]) + 1) / 2
        stego_img[j + 1] = L[i] - H[i] / 2
    
    stego_img = stego_img.reshape(len(cover_img), len(cover_img[0]))
    
    # 4. Write stego img to file
    imsave(stego_img_file, np.asarray(stego_img).astype('uint8'))

    print 'Ẩn dữ liệu thành công!'
    return True

## 4. Hàm rút trích

In [5]:
def extract_DE(stego_img_file, extracted_msg_file, recovered_cover_img):
    """
    Extracts the message from a stego image using the DE method.
    
    Parameters
    ----------
    stego_img_file : string
        The name of the stego img file.
    extracted_msg_file : string
        The name of the extracted message file.
    recovered_cover_img_file : string
        The name of the recovered cover img file.
    """
    # TODO
    # 1. Read stego img file
    stego_img = imread(stego_img_file)
    
    # 2. The decoding and authentication process (five steps)
    # 2.1. Step 1: calculate the difference values
    H = []
    L = []
    cover_img = stego_img.reshape(stego_img.size)
    for i in range(0, len(cover_img), 2):
        h = int(cover_img[i]) - int(cover_img[i + 1])
        l = (int(cover_img[i]) + int(cover_img[i + 1])) / 2
        H.append(h)
        L.append(l)
        
    # 2.2. Step 2 and 3
    # Step 2: create two disjoint sets of difference values CH and NC
    # CH: contains all changeable
    # NC: contains all nonchangeable
    # Step 3: collect LSBs of all difference values in CH and form a binary bitstream
    # In this code, we combine step 2 and 3 to one step.
    CH = []
    bitstream = [] # B
    for i in range(len(H)):
        if is_changeable(H[i], L[i]):
            CH.append(i)
            bitstream.append(H[i] % 2)
            
    # 2.3 Step 4: decode the location map from bitstream B by a JBIG2 decoder
    # and then restore the original values of differences
    # 2.3.1. decode the location map from bitstream by a JBIG2 decoder
    final_bits = bitarray.bitarray(bitstream)
    unzlib_lm = bitarray.bitarray()
    unzlib_lm.frombytes(zlib.decompress(final_bits.tobytes()))
    lm = unzlib_lm[:len(H)]
    
    # 2.3.2. restore the original values of differences
    zlib_lm = bitarray.bitarray()
    zlib_lm.frombytes(zlib.compress(lm.tobytes()))   
    s = len(zlib_lm)
    
    for i in range(len(H)):
        if i in CH:
            if lm[i] == 1:
                H[i] = H[i] / 2
            elif H[i] >= 0 and H[i] <= 1:
                H[i] = 1
            elif H[i] >= -2 and H[i] <= -1:
                H[i] = -2
            else:
                H[i] = 2 * (H[i] / 2) + bitstream[s]
                s += 1           
    
    # 2.4. Step 5: content authentication and original content restoration   
    # 2.4.1. Format extracted msg text
    # 2.4.1.1. Find index of the last bit 1 of extracted msg bit,
    # create a new string with content similar to the old string
    # but eliminate the last bits '1000 ...' based on the index of the last bit 1
    extracted_msg_bit = bitstream[s:]
    index = len(extracted_msg_bit) - 1     
    while index >= 0:
        if extracted_msg_bit[index] == 1:
            break
        index -= 1
    extracted_msg_bit = extracted_msg_bit[:index]
    
    # 2.4.1.2. Convert extracted msg bit
    extracted_msg_bit = bitarray.bitarray(extracted_msg_bit)
    extracted_msg_text = extracted_msg_bit.tobytes()
    
    # 2.4.2. Restore original content
    for i in range(len(H)):
        j = i * 2
        cover_img[j] = L[i] + (int(H[i]) + 1) / 2
        cover_img[j + 1] = L[i] - H[i] / 2
        
    cover_img = cover_img.reshape(len(stego_img), len(stego_img[0]))
    
    # 3. Write the result to file
    # 3.1. Write extracted msg text to file
    f = open(extracted_msg_file, 'w')
    f.write(extracted_msg_text)
    f.close()
    
    # 3.2. Write recovered cover img to file
    imsave(recovered_cover_img, np.asarray(cover_img).astype('uint8'))
    
    return

## 5. Chạy thí nghiệm

Bạn sẽ gọi hàm nhúng và hàm rút trích ở trên với file cover `cover.bmp` và file message `msg.txt` mà mình đính kèm. Để minh chứng hàm nhúng và hàm rút trích đã được cài đặt đúng, bạn cần mở file message ban đầu và file message được rút trích, và in nội dung của hai file này ra; ngoài ra, bạn cũng cần đọc ảnh cover ban đầu và ảnh cover khôi phục được, và so sánh hai mảng ứng với hai ảnh này với nhau (dùng hàm `np.array_equal`)

In [6]:
# TODO
input_1 = 'cover.bmp'
input_2 = 'msg.txt'
output = 'stego.bmp'

b = embed_DE(input_1, input_2, output)
if b:
    output_1 = 'extracted.txt'
    output_2 = 'recovered.bmp'
    extract_DE(output, output_1, output_2)

    f = open(input_2, 'r')
    msg_text = f.read()
    f.close()

    f = open(output_1, 'r')
    extracted_text = f.read()
    f.close()

    a = imread(input_1)
    b = imread(output_2)

    print '------------------------------'
    print msg_text
    print '------------------------------'
    print extracted_text
    print '------------------------------'
    print np.array_equal(a, b)

Kích thước của EZ + EN: 131017
Kích thước của CN: 55
Kích thước của NC: 0
Số lượng h = 1 và h = -2 của CN: 0
Chiều dài lm trước và sau khi nén: 131072 , 1264
Chiều dài của msg_bit ban đầu: 480
Ẩn dữ liệu thành công!
------------------------------
Empty your mind of all thoughts.
Let your heart be at peace.
------------------------------
Empty your mind of all thoughts.
Let your heart be at peace.
------------------------------
True


## 6. Ghi chú (nếu có)

Thời gian chạy các hàm nhúng và rút trích khá lâu (tổng thời gian xấp xỉ 9 phút)
    - Hàm nhúng: từ 4m30s đến dưới 5m.
    - Hàm rút trích: dưới 5m.

---

Enjoy DE :-)